### Initialization
* Check whether the runtime is host or local.
* Mount Google Drive when using the host runtime.

In [10]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
  runtime = "host"
except:
  runtime = "local"

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


### Parameters

In [0]:
#@title Parameters
#@markdown |Name            |Description|
#@markdown |:---            |:---|
#@markdown |`seed`|The random seed|
seed = 3984 #@param {type: "number"}

#@markdown ### `deep-coder` Repositories
#@markdown |Name            |Description|
#@markdown |:---            |:---|
#@markdown |`repository_url`|The URL of `deep-coder` git repository (enabled only in the host runtime)|
#@markdown |`branch_name`   |The branch name (enabled only in the host runtime)|
repository_url = "https://github.com/HiroakiMikami/deep-coder" #@param {type: "string"}
branch_name = "master" #@param {type: "string"}

#@markdown ### Model Parameters
#@markdown |Name               |Description|
#@markdown |:---               |:---|
#@markdown |`n_embed`          |The dimension of integer embeddings|
#@markdown |`n_units`          |The number of units in the hidden layers|
#@markdown |`num_hidden_layers`|The number of the hidden layers|
n_embed = 2 #@param {type: "number"}
n_units = 256 #@param {type: "number"}
num_hidden_layers = 3 #@param {type: "number"}

#@markdown ### Training Settings
#@markdown |Name                |Description|
#@markdown |:---                |:---|
#@markdown |`batch_size`        |The minibatch size|
#@markdown |`weight_label_false`|The weight for the loss value in the case of attribute=False. `-1` means that using the original loss function|
#@markdown |`num_epochs`        |The numer of epoch|
#@markdown |`ratio_test`        |The ratio of entries for testing|
#@markdown |`num_train`         |The number of entries used for training|
batch_size = 1 #@param {type: "number"}
weight_label_false = -1 #@param {type: "number"}
num_epochs = 100 #@param {type: "number"}
ratio_test = 0 #@param {type: "number"}
num_train = 0 #@param {type: "number"}

#@markdown ### Validation Settings
#@markdown |Name                |Description|
#@markdown |:---                |:---|
#@markdown |`timeout_second`    ||
#@markdown |`max_program_length`|The maximum length of the program|
timeout_second = 1 #@param {type: "number"}
max_program_length = 1 #@param {type: "number"}

#@markdown ### Other Settings
#@markdown |Name    |Description|
#@markdown |:---    |:---|
#@markdown |`device`|The id of GPU. `-1` means that CPU is used.|
device = 0 #@param {type: "number"}

#@markdown ### Filepath
#@markdown |Name                |Description|
#@markdown |:---                |:---|
#@markdown |`train_dataset_path`|The file path of the training dataset.|
#@markdown |`valid_dataset_path`|The file path of the validation dataset.|
#@markdown |`destination_path`  |The directory of the directory that will contain the training results.|
train_dataset_path = "/gdrive/My Drive/DeepCoder/dataset/length_1/train.pickle" #@param {type: "string"}
valid_dataset_path = "/gdrive/My Drive/DeepCoder/dataset/length_1/valid.pickle" #@param {type: "string"}
destination_path = "/gdrive/My Drive/DeepCoder/out/length_1/" #@param {type: "string"}



### Setup
* Fix the random seed
* Download the codebase
  1. Clone git repository and move to the specified branch
  2. Initialize submodule
  3. Install chainer and cupy
* Copy the dataset from Google Drive

In [0]:
import numpy as np
import random

SEED_MAX = 2**32 - 1

root_rng = np.random.RandomState(seed)
random.seed(root_rng.randint(SEED_MAX))
np.random.seed(root_rng.randint(SEED_MAX))

In [4]:
if runtime == "host":
  %cd /content
  !rm -rf deep-coder
  ![ ! -e deep-coder ] && git clone $repository_url deep-coder
  %cd deep-coder
  !git checkout origin/$branch_name
  !git submodule init
  !git submodule update
  !make -C DeepCoder_Utils/enumerative-search -j `nproc`
  !curl https://colab.chainer.org/install | sh -
  !pip install tqdm

/content
Cloning into 'deep-coder'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 1238 (delta 70), reused 64 (delta 34), pack-reused 1124
Receiving objects: 100% (1238/1238), 17.80 MiB | 2.23 MiB/s, done.
Resolving deltas: 100% (748/748), done.
/content/deep-coder
Note: checking out 'origin/master'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at cbf6f06 Add example script to generate the dataset
Submodule 'DeepCoder_Utils' (https://github.com/HiroakiMikami/DeepCoder-Utils.git) registered for path 'DeepCoder_Utils'
C

### Train DNN Model
* Create `Trainer`
* Run training

In [0]:
import pickle
import os
import chainer as ch
from chainer import datasets
from chainer.training import extensions
from src.dataset import EncodedDataset, Dataset
import src.train as T
from src.model import ModelShapeParameters

with open(train_dataset_path, "rb") as f:
    d: Dataset = pickle.load(f)
dataset = d.dataset
metadata = d.metadata
    

if num_train != 0:
    num_test = int(num_train *
                   (ratio_test if ratio_test is not None else 0.0))
    dataset, _ = datasets.split_dataset_random(
        dataset, num_train + num_test, seed=root_rng.randint(SEED_MAX))

model_shape = ModelShapeParameters(metadata, num_hidden_layers, n_embed, n_units)

n_entries = len(dataset)
dataset = EncodedDataset(Dataset(dataset, metadata))
if ratio_test is None or ratio_test == 0:
    train = dataset
    test = None
else:
    train, test = datasets.split_dataset_random(dataset, int(
        n_entries * (1.0 - ratio_test)), seed=root_rng.randint(SEED_MAX))

train_iter = ch.iterators.SerialIterator(train, batch_size)
if test is not None:
    test_iter = ch.iterators.SerialIterator(
        test, batch_size, repeat=False, shuffle=False)
else:
    test_iter = None

train = T.Training(train_iter, test_iter, destination_path, model_shape, weight_label_false,
                   num_epochs, device=device)
train.trainer.extend(extensions.LogReport())
if test_iter is not None:
    train.trainer.extend(extensions.PrintReport(
        ['epoch',
         'main/loss', 'validation/main/loss',
         'main/accuracy', 'main/accuracy_false', 'main/accuracy_true',
         'validation/main/accuracy', 'validation/main/accuracy_false', 'validation/main/accuracy_true',
         'elapsed_time']))
else:
    train.trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'main/accuracy', 'main/accuracy_false', 'main/accuracy_true', 'elapsed_time']))


In [6]:
train.trainer.run()

epoch       main/loss   main/accuracy  main/accuracy_false  main/accuracy_true  elapsed_time
1           0.299507    0.90571        0.954044             0.0294118           9.53025       
2           0.190456    0.948097       1                    0                   9.7745        
3           0.189035    0.948097       1                    0                   10.0176       
4           0.18964     0.948097       1                    0                   10.257        
5           0.187792    0.948097       1                    0                   10.503        
6           0.187395    0.948097       1                    0                   10.7499       
7           0.182942    0.948097       1                    0                   10.9909       
8           0.17956     0.948097       1                    0                   11.2342       
9           0.176796    0.949827       0.996324             0.0882353           11.5084       
10          0.170745    0.951557       0.999081     

### Save DNN Model

In [0]:
import os
import chainer as ch

if not os.path.exists(destination_path):
    os.makedirs(destination_path)

with open(os.path.join(destination_path, "model-shape.pickle"), "wb") as f:
    pickle.dump(model_shape, f)

ch.serializers.save_npz(os.path.join(destination_path, "model.npz"), train.predictor)


### Validate DNN Model

In [8]:
import pickle
import os
import chainer as ch
from chainer import datasets
from src.dataset import EncodedDataset, Dataset
import src.inference as I
from src.model import ModelShapeParameters
from tqdm import tqdm_notebook as tqdm

model = I.InferenceModel(model_shape)
ch.serializers.load_npz(os.path.join(destination_path, "model.npz"), model.predictor)

with open(valid_dataset_path, "rb") as f:
    dataset: Dataset = pickle.load(f)

pred = I.predict_with_neural_network(model_shape, model)

results = dict([])
num_succ = 0
for i, (entry,) in enumerate(tqdm(dataset.dataset)):
    result = I.search(
        os.path.join(os.getcwd(), "DeepCoder_Utils",
                     "enumerative-search", "search"),
        timeout_second,
        model_shape.dataset_metadata.value_range,
        entry.examples,
        max_program_length,
        pred
    )
    results[i] = result
    if result.is_solved:
        num_succ += 1

print("Solved: {} of {} examples".format(num_succ, len(dataset.dataset)))



Solved: 5 of 5 examples


### Save Validation Result

In [0]:
import os

if not os.path.exists(destination_path):
    os.makedirs(destination_path)

with open(os.path.join(destination_path, "result.pickle"), "wb") as f:
    pickle.dump(results, f)